In [ ]:
# 튜토리얼 진행을 위한 모듈 import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image
import warnings

np.set_printoptions(suppress=True, precision=3)

# 경고 메시지 출력 표기 생략
warnings.filterwarnings('ignore')

# 회귀 (regression)

회귀 분석(regression analysis)은 관찰된 연속형 변수들에 대해 두 변수 사이의 모형을 구한뒤 적합도를 측정해 내는 분석 방법입니다.

하나의 종속변수와 하나의 독립변수 사이의 관계를 분석할 경우를 **단순회귀분석(simple regression analysis)**, 하나의 종속변수와 여러 독립변수 사이의 관계를 규명하고자 할 경우를 **다중회귀분석(multiple regression analysis)**이라고 합니다.

**예시**
- 주택 가격 예측
- 매출액 예측
- 주가 예측
- 온도 예측

대표적인 회귀 모델로는

- 최소제곱법(Ordinary Least Squares)을 활용한 **LinearRegression**
- 경사하강법(Gradient Descent)을 활용한 **SGDRegressor**
- 선형 회귀 모델에 L1, L2 규제를 더한 **Lasso, Ridge, ElasticNet**

등이 있습니다.

[도큐먼트](https://scikit-learn.org/stable/supervised_learning.html#supervised-learning)

## 회귀 모델을 위한 평가 지표

In [ ]:
def make_linear(w=0.5, b=0.8, size=50, noise=1.0):
    np.random.seed(123)
    x = np.arange(size)
    y = w * x + b
    noise = np.random.uniform(-abs(noise), abs(noise), size=y.shape)
    yy = y + noise
    plt.figure(figsize=(10, 7))
    plt.plot(x, y, color='r', label=f'y = {w}*x + {b}')
    plt.scatter(x, yy, label='data')
    plt.legend(fontsize=20)
    plt.show()
    print(f'w: {w}, b: {b}')
    return x, y, yy

In [ ]:
x, y_true, y_pred = make_linear(size=50, w=1.5, b=0.8, noise=5.5)

### MSE (Mean Squared Error)

- 예측 값과 실제 값의 차이에 대한 **제곱**에 대하여 평균을 낸 값
- MSE 오차가 작으면 작을수록 좋지만, 과대적합이 될 수 있음에 주의합니다.
- 예측 값과 실제 값보다 크게 예측이 되는지 작게 예측되는지 알 수 없습니다.

$\Large MSE = \frac{1}{n}\sum_{i=1}^{n}{(y_i - \hat{y_i})^2}$

Python 코드로 위의 수식을 그대로 구현합니다.

`mse` 변수에 결과를 대입합니다.

In [ ]:
# 코드를 입력해 주세요
mse = 

In [ ]:
# 코드검증
print('mse = {:.3f}'.format(mse))

`sklearn.metrics` 패키지에 `mean_squared_error`를 활용합니다.

In [ ]:
from sklearn.metrics import mean_squared_error

`mse_` 변수에 결과를 대입합니다.

In [ ]:
# 코드를 입력해 주세요
mse_ = 

In [ ]:
# 코드검증
print('mse = {:.3f}'.format(mse_))

### MAE (Mean Absolute Error)

- 예측값과 실제값의 차이에 대한 **절대값**에 대하여 평균을 낸 값
- 실제 값과 예측 값 차이를 절대 값으로 변환해 평균을 계산합니다. 작을수록 좋지만, 과대적합이 될 수 있음에 주의합니다.
- **스케일에 의존적**입니다.

예를 들어, 아파트 집값은 10억, 20억으로 구성되어 있고, 과일 가격은 5000원, 10000원으로 구성되어 있을때,

예측하는 각각 모델의 MSE 가 똑같이 100 이 나왔다고 가정한다며,동일한 오차율이 아님에도 불구하고 동일하게 평가되어 지는 현상이 발생합니다.
이는 MSE 오차에서도 마찬가지 입니다.

$\Large MAE = \frac{1}{n}\sum_{i=1}^{n}{|(y_i - \hat{y_i})|}$

Python 코드로 위의 수식을 그대로 구현합니다.

`mae` 변수에 결과를 대입합니다.

In [ ]:
# 코드를 입력해 주세요
mae = 

In [ ]:
# 코드검증
print('mae = {:.3f}'.format(mae))

`sklearn.metrics` 패키지에 `mean_absolute_error`를 활용합니다.

In [ ]:
from sklearn.metrics import mean_absolute_error

`mae_` 변수에 결과를 대입합니다.

In [ ]:
# 코드를 입력해 주세요
mae_ = 

In [ ]:
# 코드검증
print('mae = {:.3f}'.format(mae_))

## 회귀 모델 (Regression Models)

### 모델별 성능 확인을 위한 함수

In [ ]:
# 모듈 설치
!pip install --upgrade teddynote -q

In [ ]:
from teddynote import utils

# 그래프 사이즈 설정
utils.set_plot_options(figsize=(12, 10))

# MSE 에러 설정
utils.set_plot_error('mse')

### 보스턴 집 값 데이터

**데이터 로드 (load_boston)**

In [ ]:
from sklearn.datasets import load_boston

# 코드를 입력해 주세요
data = load_boston()

df = pd.DataFrame(data['data'], columns=data['feature_names'])
df['target'] = data['target']
df.head()

**컬럼 소개**

속성 수 : 13

* **CRIM**: 자치시 별 범죄율
* **ZN**: 25,000 평방 피트를 초과하는 주거용 토지의 비율
* **INDUS**: 비소매(non-retail) 비즈니스 토지 비율
* **CHAS**: 찰스 강과 인접한 경우에 대한 더비 변수 (1= 인접, 0= 인접하지 않음)
* **NOX**: 산화 질소 농도 (10ppm)
* **RM**:주택당 평균 객실 수
* **AGE**: 1940 년 이전에 건축된 자가소유 점유 비율
* **DIS**: 5 개의 보스턴 고용 센터까지의 가중 거리     
* **RAD**: 고속도로 접근성 지수
* **TAX**: 10,000 달러 당 전체 가치 재산 세율
* **PTRATIO**  도시별 학생-교사 비율
* **B**: 인구당 흑인의 비율. 1000(Bk - 0.63)^2, (Bk는 흑인의 비율을 뜻함)
* **LSTAT**: 하위 계층의 비율
* **target**: 자가 주택의 중앙값 (1,000 달러 단위)

**학습(train) / 테스트(test)** 용 데이터를 분할 합니다.

In [ ]:
from sklearn.model_selection import train_test_split

# 시드 설정
SEED=30

In [ ]:
# 코드를 입력해 주세요
x_train, x_test, y_train, y_test = 

`x_train`, `x_test`의 shape를 출력합니다.

In [ ]:
# 코드를 입력해 주세요
x_train.shape, x_test.shape

In [ ]:
# 검증코드
x_train.head()

## LinearRegression

[도큐먼트](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
# 코드를 입력해 주세요
model = 

In [ ]:
# 코드를 입력해 주세요
model.

`pred` 변수에 예측 결과를 대입합니다.

In [ ]:
# 코드를 입력해 주세요
pred = 

In [ ]:
utils.plot_error('LinearRegression', y_test, pred)

## 규제 (Regularization)

학습이 과대적합 되는 것을 방지하고자 일종의 **penalty**를 부여하는 것

**L2 규제 (L2 Regularization)**

* 각 가중치 제곱의 합에 규제 강도(Regularization Strength) α를 곱한다. 
* α를 크게 하면 가중치가 더 많이 감소되고(규제를 중요시함), α를 작게 하면 가중치가 증가한다(규제를 중요시하지 않음).

**L1 규제 (L1 Regularization)**

* 가중치의 제곱의 합이 아닌 **가중치의 합**을 더한 값에 규제 강도(Regularization Strength) α를 곱하여 오차에 더한다. 
* 어떤 가중치(w)는 실제로 0이 된다. 즉, 모델에서 완전히 제외되는 특성이 생기는 것이다. 


**L2 규제가 L1 규제에 비해 더 안정적이라 일반적으로는 L2규제가 더 많이 사용된다**

### Ridge (L2 Regularization)

- L2 규제 계수를 적용합니다. 
- 선형회귀에 가중치 (weight)들의 제곱합에 대한 최소화를 추가합니다.

**주요 hyperparameter**
- `alpha`: 규제 계수

**수식**

`α`는 규제 계수(강도)를 의미합니다.

$\Large Error=MSE+α\sum_{i=1}^{n}{w_i^2}$ 

In [ ]:
from sklearn.linear_model import Ridge

**규제 계수(alpha)**를 정의합니다.

In [ ]:
# 값이 커질 수록 큰 규제입니다.
alphas = [100, 10, 1, 0.1, 0.01, 0.001, 0.0001]

In [ ]:
for alpha in alphas:
    # 코드를 입력해 주세요
    ridge = 
    ridge.
    pred = 
    utils.add_error('Ridge(alpha={})'.format(alpha), y_test, pred)
utils.plot_all()

coef_는 **feature의 가중치**를 보여줍니다.

가중치(weight)를 토대로 회귀 예측시 어떤 feature가 주요하게 영향을 미쳤는지 보여 줍니다.

In [ ]:
x_train.columns

In [ ]:
ridge.coef_

**DataFrame**으로 feature별 가중치를 시각화 합니다.

In [ ]:
def plot_coef(columns, coef):
    coef_df = pd.DataFrame(list(zip(columns, coef)))
    coef_df.columns=['feature', 'coef']
    coef_df = coef_df.sort_values('coef', ascending=False).reset_index(drop=True)
    
    fig, ax = plt.subplots(figsize=(9, 7))
    ax.barh(np.arange(len(coef_df)), coef_df['coef'])
    idx = np.arange(len(coef_df))
    ax.set_yticks(idx)
    ax.set_yticklabels(coef_df['feature'])
    fig.tight_layout()
    plt.show()

In [ ]:
plot_coef(x_train.columns, ridge.coef_)

이번에는, **alpha 값에 따른 coef 의 차이**를 확인해 봅시다

In [ ]:
ridge_100 = Ridge(alpha=100)
ridge_100.fit(x_train, y_train)
ridge_pred_100 = ridge_100.predict(x_test)

ridge_01 = Ridge(alpha=0.1)
ridge_01.fit(x_train, y_train)
ridge_pred_01 = ridge_01.predict(x_test)

In [ ]:
plot_coef(x_train.columns, ridge_100.coef_)

In [ ]:
plot_coef(x_train.columns, ridge_01.coef_)

### Lasso (L1 Regularization)

Lasso(Least Absolute Shrinkage and Selection Operator)

- 선형 회귀에 L1 규제 계수를 적용합니다.
- 가중치(weight)의 절대 값의 합을 최소화 하는 계수를 추가 합니다.
- 불필요한 회귀 계수를 급격히 감소, 0으로 만들어 제거합니다.
- 특성(Feature) 선택에 유리합니다.

**주요 hyperparameter**
- `alpha`: L1 규제 계수

**수식**  

`α`는 규제 계수(강도)를 의미합니다.

$\Large Error=MSE+α\sum_{i=1}^{n}{|w_i|}$

In [ ]:
from sklearn.linear_model import Lasso

In [ ]:
# 값이 커질 수록 큰 규제입니다.
alphas = [100, 10, 1, 0.1, 0.01, 0.001, 0.0001]

In [ ]:
for alpha in alphas:
    # 코드를 입력해 주세요
    lasso = 
    lasso.
    pred = 
    utils.add_error('Lasso(alpha={})'.format(alpha), y_test, pred)
utils.plot_all()

In [ ]:
lasso_100 = Lasso(alpha=100)
lasso_100.fit(x_train, y_train)
lasso_pred_100 = lasso_100.predict(x_test)

lasso_001 = Lasso(alpha=0.01)
lasso_001.fit(x_train, y_train)
lasso_pred_001 = lasso_001.predict(x_test)

In [ ]:
plot_coef(x_train.columns, lasso_001.coef_)

In [ ]:
lasso_001.coef_

Lasso 모델에 너무 큰 alpha 계수를 적용하면 **대부분의 feature들의 가중치가 0으로 수렴**합니다.

In [ ]:
plot_coef(x_train.columns, lasso_100.coef_)

In [ ]:
lasso_100.coef_

### ElasticNet

Elastic Net 회귀모형은 **가중치의 절대값의 합(L1)과 제곱합(L2)을 동시에** 제약 조건으로 가지는 모형입니다.

In [ ]:
Image(url='https://miro.medium.com/max/1312/1*j_DDK7LbVrejTq0tfmavAA.png', width=500)

**주요 hyperparameter**

`alpha`: 규제 계수

`l1_ratio (default=0.5)`

- l1_ratio = 0 (L2 규제만 사용). 
- l1_ratio = 1 (L1 규제만 사용). 
- 0 < l1_ratio < 1 (L1 and L2 규제의 혼합사용)

In [ ]:
from sklearn.linear_model import ElasticNet

In [ ]:
alpha=0.01
ratios = [0.2, 0.5, 0.8]

In [ ]:
for ratio in ratios:
    # 코드를 입력해 주세요
    elasticnet = 
    elasticnet.
    pred = 
    utils.add_error('ElasticNet(l1_ratio={})'.format(ratio), y_test, pred)
utils.plot_all()

In [ ]:
elsticnet_20 = ElasticNet(alpha=0.01, l1_ratio=0.2)
elsticnet_20.fit(x_train, y_train)
elasticnet_pred_20 = elsticnet_20.predict(x_test)

elsticnet_80 = ElasticNet(alpha=0.01, l1_ratio=0.8)
elsticnet_80.fit(x_train, y_train)
elasticnet_pred_80 = elsticnet_80.predict(x_test)

In [ ]:
plot_coef(x_train.columns, elsticnet_20.coef_)

In [ ]:
plot_coef(x_train.columns, elsticnet_80.coef_)

## Scaler 적용

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

### MinMaxScaler (정규화)

정규화 (Normalization)도 표준화와 마찬가지로 데이터의 스케일을 조정합니다.

정규화가 표준화와 다른 가장 큰 특징은 **모든 데이터가 0 ~ 1 사이의 값**을 가집니다.

즉, 최대값은 1, 최소값은 0으로 데이터의 범위를 조정합니다.

**min값과 max값을 0~1사이로 정규화**

In [ ]:
# 코드를 입력해 주세요 (x_train만 변환)
minmax_scaler = 
minmax_scaled = 

In [ ]:
# 코드검증
round(pd.DataFrame(minmax_scaled).describe(), 2)

### StandardScaler (표준화)

표준화는 데이터의 **평균을 0 분산 및 표준편차를 1**로 만들어 줍니다.

**표준화를 하는 이유**

- 서로 **다른 통계 데이터들을 비교하기 용이**하기 때문입니다.
- 표준화를 하면 평균은 0, 분산과 표준편차는 1로 만들어 데이터의 **분포를 단순화 시키고, 비교를 용이**하게 합니다.

$\Large z = \frac{(X - \mu)}{\sigma}$

In [ ]:
# 코드를 입력해 주세요 (x_train만 변환)
std_scaler = 
std_scaled = 

In [ ]:
round(pd.DataFrame(std_scaled).describe(), 2)

## 파이프라인 (pipeline)

scikit-learn의 **전처리(pre-processing)용 모듈과 모델의 학습 기능을 파이프라인으로 합칠 수** 있습니다.

- 파이프라인으로 결합된 모형은 원래의 모형이 가지는 `fit`, `predict` 함수를 가집니다.
- 파이프라인에 정의된 순서에 따라 전처리 모듈이 먼저 호출되어 전처리 과정을 거친 후 모델이 학습하게 됩니다.

[Pipeline 공식 도큐먼트](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html)

In [ ]:
from sklearn.pipeline import make_pipeline

- `MinMaxScaler`를 적용
- ElasticNet, `alpha=0.1`, `l1_ratio=0.2` 적용

In [ ]:
# 코드를 입력해 주세요
pipeline = make_pipeline(
    
    
)

In [ ]:
# 코드를 입력해 주세요
pipeline.
pipeline_pred = 

In [ ]:
utils.plot_error('MinMax ElasticNet', y_test, pipeline_pred)

- `StandardScaler`를 적용
- ElasticNet, `alpha=0.1`, `l1_ratio=0.2` 적용

In [ ]:
# 코드를 입력해 주세요
pipeline = make_pipeline(
    
    
)

In [ ]:
# 코드를 입력해 주세요
pipeline.
pipeline_pred = 

In [ ]:
utils.plot_error('Standard ElasticNet', y_test, pipeline_pred)

### Polynomial Features

다항식의 계수간 상호작용을 통해 **새로운 feature를 생성**합니다.

예를들면, [a, b] 2개의 feature가 존재한다고 가정하고,

degree=2로 설정한다면, polynomial features 는 [1, a, b, a^2, ab, b^2] 가 됩니다.

**주의**
- `degree`를 올리면, 기하급수적으로 많은 feature 들이 생겨나며, 학습 데이터에 지나치게 과대적합 될 수 있습니다.

**주요 hyperparameter**

- `degree`: 차수
- `include_bias`: 1로 채운 컬럼 추가 여부

[Polynomial Features 도큐먼트](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html?highlight=poly%20feature#sklearn.preprocessing.PolynomialFeatures)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
x = np.arange(5).reshape(-1, 1)
x

`degree=2`, `include_bias=False` 인 경우

In [ ]:
poly = PolynomialFeatures(degree=2, include_bias=False)
x_poly = poly.fit_transform(x)
x_poly

`degree=2`, `include_bias=True` 인 경우

In [ ]:
poly = PolynomialFeatures(degree=2, include_bias=True)
x_poly = poly.fit_transform(x)
x_poly

`degree=3`, `include_bias=True` 인 경우

In [ ]:
poly = PolynomialFeatures(degree=3, include_bias=True)
x_poly = poly.fit_transform(x)
x_poly

보스톤 집 값 데이터의 features에 `PolynomialFeatures`를 적용합니다.

In [ ]:
poly = PolynomialFeatures(degree=2, include_bias=False)
poly_features = poly.fit_transform(x_train)[0]
poly_features

In [ ]:
x_train_poly = poly.fit_transform(x_train)
x_train_poly[0]

In [ ]:
pd.DataFrame(x_train_poly, columns=poly.get_feature_names()).head()

`PolynomialFeature`도 파이프라인(pipeline)을 활용하여 전처리 해준다면, 손쉽게 구현 및 적용이 가능합니다.

- `PolynomialFeatures: degree=2, include_bias=False` 지정
- `ElasticNet: alpha=0.1, l1_ratio=0.2` 지정

In [ ]:
poly_pipeline = make_pipeline(
    # 코드를 입력해 주세요
    
    
)

In [ ]:
# 코드를 입력해 주세요
poly_pipeline.
poly_pred = 

In [ ]:
utils.plot_error('Poly ElasticNet', y_test, poly_pred)